In [71]:
import pandas as pd

arribos = pd.read_csv("arribos_servicio_clean (1).csv")
arribos


,id,diaSemana,arribo,inicioAtencionCaja,finPedido,finPago,metodoPago,observacion
0,o1pzIp49yzu3qmR5AfCt,viernes,"13:26:54,618","13:26:56,699","13:27:08,380","13:27:17,532",efectivo,NaN
1,HGkM4bWI6hnnnpuWDYPz,viernes,"13:16:40,605","13:16:50,280","13:16:59,991","13:17:23,559",efectivo,NaN
2,w8ri1AlcTa3jmgMBNxqe,viernes,"14:55:50,438","14:56:01,781","14:56:34,875","14:57:40,297",efectivo,NaN
3,Tz4dyCJaAKmtrHwgPIJ7,viernes,"13:35:23,584","13:36:40,562","13:36:55,170","13:37:33,919",efectivo,NaN
4,7rKAfKzMmbd93GwIJzCo,viernes,"13:52:37,752","13:52:41,177","13:52:59,928","13:53:04,995",efectivo,NaN
...,...,...,...,...,...,...,...,...
620,PpuM986QfY01LlGh1B5y,domingo,"14:13:40,303","14:17:02,164","14:17:27,766","14:18:14,869",efectivo,NaN
621,PlxhS9A8cE1rYYX42RCH,domingo,"12:42:41,405","12:44:09,010","12:44:20,270","12:44:28,085",efectivo,NaN
622,Ph4okJnBvHgWsZv1bFML,domingo,"14:41:28,666","14:47:27,793","14:47:35,953","14:48:40,364",Tarjeta,NaN
623,PWA9QTBF4yjJEyYDzFRe,domingo,"12:10:18,932","12:10:27,702","12:10:30,532","12:10:41,760",efectivo,NaN


In [72]:
cols_hora = ['arribo', 'inicioAtencionCaja', 'finPedido', 'finPago']
for c in cols_hora:
    arribos[c] = pd.to_datetime(arribos[c], format='%H:%M:%S,%f').dt.time

# 2. Filtrar viernes y quedarnos solo con las columnas de hora
df_viernes = (
    arribos[arribos['diaSemana'] == 'viernes']
      [cols_hora]
      .reset_index(drop=True)
)

# 3. Filtrar sábado y domingo, y quedarnos sólo con las columnas de hora
df_finde = (
    arribos[arribos['diaSemana'].isin(['sabado', 'sábado', 'domingo'])]
      [cols_hora]
      .reset_index(drop=True)
)


In [73]:
df_viernes

,arribo,inicioAtencionCaja,finPedido,finPago
0,13:26:54.618000,13:26:56.699000,13:27:08.380000,13:27:17.532000
1,13:16:40.605000,13:16:50.280000,13:16:59.991000,13:17:23.559000
2,14:55:50.438000,14:56:01.781000,14:56:34.875000,14:57:40.297000
3,13:35:23.584000,13:36:40.562000,13:36:55.170000,13:37:33.919000
4,13:52:37.752000,13:52:41.177000,13:52:59.928000,13:53:04.995000
...,...,...,...,...
85,12:53:06.740000,12:53:13.432000,12:53:21.663000,12:54:26.916000
86,14:04:39.712000,14:05:03.504000,14:05:04.203000,14:05:11.055000
87,14:59:34.077000,14:59:40.352000,15:00:02.419000,15:00:18.958000
88,13:35:07.174000,13:36:28.405000,13:36:30.323000,13:36:35.974000


In [74]:
df_finde

,arribo,inicioAtencionCaja,finPedido,finPago
0,14:46:24.384000,14:47:50.555000,14:48:03.816000,14:48:12.413000
1,13:49:48.155000,13:57:03.267000,13:57:34.055000,13:57:41.375000
2,14:33:42.077000,14:35:29.121000,14:35:55.771000,14:36:03.975000
3,14:48:28.739000,14:51:05.376000,14:51:13.084000,14:51:35.340000
4,12:24:13.963000,12:24:18.049000,12:24:24.888000,12:24:35.871000
...,...,...,...,...
530,14:13:40.303000,14:17:02.164000,14:17:27.766000,14:18:14.869000
531,12:42:41.405000,12:44:09.010000,12:44:20.270000,12:44:28.085000
532,14:41:28.666000,14:47:27.793000,14:47:35.953000,14:48:40.364000
533,12:10:18.932000,12:10:27.702000,12:10:30.532000,12:10:41.760000


In [75]:
# 3. Función para convertir strings de hora en Timestamps
def prepare_times(df):
    df = df.copy()
    for col in cols_hora:
        s = df[col].astype(str).str.replace(',', '.', regex=False)
        dt1 = pd.to_datetime(s, format='%H:%M:%S.%f', errors='coerce')
        dt2 = pd.to_datetime(s, format='%H:%M:%S',   errors='coerce')
        df[col + '_dt'] = dt1.fillna(dt2)
    return df

# 4. Función para resumen horario.
#    n_days = número de días que comprenden los datos,
#    para dividir y obtener un promedio diario si n_days > 1.
def hourly_summary(df, n_days=1):
    df = prepare_times(df)
    hours = list(range(11, 16))                   # intervalos 11–12, …, 15–16
    labels = [f"{h:02d}:00" for h in hours]

    # Hora de arribo y fin de pago
    df['h_arr'] = df['arribo_dt'].dt.hour
    df['h_sal'] = df['finPago_dt'].dt.hour

    # Conteos por hora
    arribos = df.groupby('h_arr').size().reindex(hours, fill_value=0)
    salidas = df.groupby('h_sal').size().reindex(hours, fill_value=0)

    # Construir DataFrame de resumen
    summary = pd.DataFrame({
        'arribos': arribos.values,
        'salidas': salidas.values
    }, index=labels)
    summary['cum_arribos'] = summary['arribos'].cumsum()
    summary['cum_salidas'] = summary['salidas'].cumsum()
    summary['en_sistema']  = summary['cum_arribos'] - summary['cum_salidas']

    # Si son datos de múltiples días, dividir para promedio diario
    if n_days > 1:
        summary = summary / n_days
        summary = summary.rename(columns=lambda c: f"{c}_promedio")
    return summary

# 5. Generar los resultados usando n_days=2 para dos viernes,
#    y n_days=4 para dos sábados + dos domingos
res_viernes     = hourly_summary(df_viernes, n_days=2)
res_finde_avg   = hourly_summary(df_finde,   n_days=4)

In [76]:
res_viernes

,arribos_promedio,salidas_promedio,cum_arribos_promedio,cum_salidas_promedio,en_sistema_promedio
11:00,0.0,0.0,0.0,0.0,0.0
12:00,4.0,4.0,4.0,4.0,0.0
13:00,20.5,20.0,24.5,24.0,0.5
14:00,20.5,20.5,45.0,44.5,0.5
15:00,0.0,0.5,45.0,45.0,0.0


In [77]:
res_finde_avg

,arribos_promedio,salidas_promedio,cum_arribos_promedio,cum_salidas_promedio,en_sistema_promedio
11:00,7.25,6.50,7.25,6.50,0.75
12:00,25.50,25.00,32.75,31.50,1.25
13:00,48.00,45.50,80.75,77.00,3.75
14:00,51.50,51.50,132.25,128.50,3.75
15:00,1.50,5.25,133.75,133.75,0.00


In [78]:
productos = pd.read_csv("productos_limpio.csv")
productos

,id,copas,gofres,bebidas,crepes,diaSemana,observacion,helados
0,0KNVXuPCDAkUaB8ZV9Fi,[],[],[],[],viernes,NaN,"[{'inicio': '14:27:09,739', 'fin': '14:27:40,0..."
1,0PUBZu6abxga7eqgYlmI,[],[],[],[],domingo,NaN,"[{'inicio': '13:23:46,397', 'fin': '13:24:18,5..."
2,0Yp0BA1puO3Rxr7NMdEy,[],[],[],[],domingo,NaN,"[{'inicio': '13:41:12,604', 'fin': '13:41:26,0..."
3,0eOmEWZGcP7bZlPXZF0h,[],[],[],[],sábado,NaN,"[{'inicio': '13:58:57,860', 'fin': '13:59:21,3..."
4,0heZFJMic03pxxu3iWln,[],[],"[{'inicio': '13:10:53,426', 'fin': '13:12:45,2...",[],domingo,!Fueron dos malteadas,[]
...,...,...,...,...,...,...,...,...
519,zqy4MfZxlRdI51u9V8lC,[],[],[],[],domingo,NaN,"[{'inicio': '14:20:05,375', 'fin': '14:20:24,9..."
520,ztOsJ1Vzd7Z6NLAXZAb0,[],[],[],[],sábado,NaN,"[{'inicio': '14:30:12,471', 'fin': '14:30:31,8..."
521,zveavgvE9vAFkJqtfdVR,[],[],[],[],domingo,NaN,"[{'inicio': '13:24:50,635', 'fin': '13:25:23,3..."
522,zxQb72zK2RY1RQPC0Zlz,[],[],[],[],viernes,NaN,"[{'inicio': '15:00:18,570', 'fin': '15:01:21,7..."


In [79]:
import pandas as pd
import ast

# Columnas que contienen las listas (o su representación en string)
cols_items = ['copas', 'gofres', 'bebidas', 'crepes', 'helados']

def extract_events(df, item_cols):
    events = []
    for _, row in df.iterrows():
        for col in item_cols:
            cell = row[col]
            # Saltar NaN
            if pd.isna(cell):
                continue
            # Si es string, intentar convertirlo a objeto Python
            if isinstance(cell, str):
                try:
                    cell = ast.literal_eval(cell)
                except Exception:
                    continue
            # Ahora debería ser lista
            if not isinstance(cell, list):
                continue
            for itm in cell:
                if not isinstance(itm, dict):
                    continue
                inicio = itm.get('inicio', '').replace(',', '.')
                fin    = itm.get('fin',    '').replace(',', '.')
                # Parseo robusto de timestamps
                dt_i = pd.to_datetime(inicio, format='%H:%M:%S.%f', errors='coerce')
                if pd.isna(dt_i):
                    dt_i = pd.to_datetime(inicio, format='%H:%M:%S', errors='coerce')
                dt_f = pd.to_datetime(fin, format='%H:%M:%S.%f', errors='coerce')
                if pd.isna(dt_f):
                    dt_f = pd.to_datetime(fin, format='%H:%M:%S', errors='coerce')
                events.append({'inicio': dt_i, 'fin': dt_f})
    return pd.DataFrame(events)

def hourly_active_orders(events, hours, n_days=1):
    rows = []
    for h in hours:
        start = pd.Timestamp(year=1900, month=1, day=1, hour=h)
        end   = start + pd.Timedelta(hours=1)
        mask  = (events['inicio'] < end) & (events['fin'] > start)
        count = mask.sum()
        avg   = count / n_days
        rows.append({
            'hora': f'{h:02d}:00',
            'pedidos_totales': count,
            'pedidos_promedio': avg
        })
    return pd.DataFrame(rows).set_index('hora')

# --- Ejecución (asegúrate de que 'df' es tu DataFrame original) ---
events_df     = extract_events(productos, cols_items)
hours         = list(range(11, 16))  # 11–12, …, 15–16
n_days_total  = 6                    # 2 viernes + 2 sábados + 2 domingos

resumen_pedidos = hourly_active_orders(events_df, hours, n_days=n_days_total)


In [80]:
resumen_pedidos

,pedidos_totales,pedidos_promedio
hora,,
11:00,17,2.833333
12:00,120,20.000000
13:00,343,57.166667
14:00,331,55.166667
15:00,36,6.000000
